# Installation

In [3]:
%pip install --upgrade langchain langchain-community langchainhub langchain-openai chromadb unstructured[all-docs] ollama lamini

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 689.5/689.5 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
!pip -q install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 14.5 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
import pandas as pd

In [4]:
from dotenv import load_dotenv
import lamini
import os

# load_dotenv("/content/drive/MyDrive/DATA298/.env")
# lamini.api_key = os.environ["LAMINI_API_KEY"]


In [5]:
import json
import re
from pprint import pprint
import random
import numpy as np
from datetime import datetime
import pandas as pd
import torch
from datasets import Dataset, load_dataset

In [6]:
OPENAI_API_KEY = 'sk-proj-Thg8IdveW3PaOqjUmPl7T3BlbkFJNFqz886rEIrU5gnDLGoo'

# Pre-processing

In [7]:
def load_data(file_path):
    try:
        return pd.read_csv(file_path)
    except Exception as e:
        print(f"Failed to load data: {e}")
        return pd.DataFrame()

def remove_missing_data(df):
    initial_size = len(df)
    df_clean = df.dropna()
    final_size = len(df_clean)
    print(f"Removed {initial_size - final_size} rows with missing values.")
    return df_clean

def filter_character_data(df, character_name):
    return df[df['Character'] == character_name]

def export_dialogues(df, file_name):
    try:
        df['Dialogue'].to_csv(file_name, index=False)
        print(f"Data exported successfully to {file_name}")
    except Exception as e:
        print(f"Failed to export data: {e}")

def check_data_consistency(data, dataset_name):
    if isinstance(data, Dataset):
        df = data.to_dataframe()
    else:
        df = data

    print(f"Checking consistency for {dataset_name}...")
    print(f"Total entries in {dataset_name}: {len(df)}")

    missing_values_count = df.isnull().sum().sum()
    if missing_values_count > 0:
        print(f"{dataset_name} has {missing_values_count} missing values.")
    else:
        print(f"{dataset_name} is complete without any missing values.")


DEFAULT_SYSTEM_PROMPT = """
<<SYS>>
You are an expert at impersonating celebrities. In particular, you are
a world-leading expert at answering questions like Jerry from the hit
sitcom Seinfield.
<</SYS>>

<<INST>>
Generate a response that sounds as close to what Jerry would say. I have
included some relevant samples of what he actually said in the show.
<</INST>>
""".strip()

def generate_training_prompt(question, response, retriever, system_prompt=DEFAULT_SYSTEM_PROMPT):
    return f"""### Instruction: {system_prompt}

### Input:
{question.strip()}

### Context:
{custom_format(retriever.invoke(question))}

### Response:
{response}
""".strip()

def custom_format(docs):
  output = ""
  for doc in docs:
    doc = doc.page_content.split("Dialogue:")[-1]
    output += doc
    output += "\n\n"
  return output

In [8]:
def generate_training_data(file_path, character_name):
    print("Loading data from file...")
    df = load_data(file_path)
    if df.empty:
        print("Loaded DataFrame is empty or file not found. Exiting...")
        return None, None, None

    if 'Character' not in df.columns or 'Dialogue' not in df.columns:
        print("Essential columns are missing from the DataFrame. Exiting...")
        return None, None, None

    print(f"Filtering data for character: {character_name}...")
    character_df = filter_character_data(df, character_name)
    dialogue_file = f"{character_name.lower()}_dialogue.csv"
    print(f"Exporting dialogues to {dialogue_file}...")
    export_dialogues(character_df, dialogue_file)

    if character_df.empty:
        print(f"No dialogues for character {character_name}. Exiting...")
        return None, None, None

    print(f"Processing dialogues for character: {character_name}...")
    try:
        loader = CSVLoader(dialogue_file)
        docs = loader.load()
    except Exception as e:
        print(f"Failed to load dialogues from {dialogue_file}: {e}")
        return None, None, None

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)
    seinfeld_db = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(api_key=OPENAI_API_KEY))
    retriever = seinfeld_db.as_retriever(search_kwargs={'k': 10})

    print("Compiling question-answer pairs...")
    questions, answers = [], []
    character_list = df['Character'].tolist()
    dialogue_list = df['Dialogue'].tolist()

    for i in range(1, len(character_list)):
        if character_list[i] == character_name and character_list[i-1] != character_name:
            question = dialogue_list[i-1]
            answer = dialogue_list[i]
            if isinstance(question, str) and isinstance(answer, str):
                questions.append(question)
                answers.append(answer)
            else:
                if not isinstance(question, str):
                    print(f"Skipping non-string question at index {i-1}. Found type: {type(question).__name__}")
                if not isinstance(answer, str):
                    print(f"Skipping non-string answer at index {i}. Found type: {type(answer).__name__}")

    print("Post-compilation consistency check...")
    check_data_consistency(pd.DataFrame({'Question': questions, 'Answer': answers}), "Q&A Pairs")

    print("Generating training prompts...")
    tmpList = []
    for j, (question, answer) in enumerate(zip(questions, answers)):
        if isinstance(question, str) and isinstance(answer, str):
            prompt = generate_training_prompt(question.strip(), answer.strip(), retriever)
            tmp_dict = {"question": question, "answer": answer, "training_prompt": prompt}
            tmpList.append(tmp_dict)

    print("Split data into train, test, and val sets...")
    random.shuffle(tmpList)
    train_cutoff = int(0.8 * len(tmpList))
    test_cutoff = train_cutoff + int(0.1 * len(tmpList))

    train_data = tmpList[:train_cutoff]
    test_data = tmpList[train_cutoff:test_cutoff]
    val_data = tmpList[test_cutoff:]

    print("Final consistency check...")
    train_data_df = pd.DataFrame(train_data)
    test_data_df = pd.DataFrame(test_data)
    val_data_df = pd.DataFrame(val_data)

    check_data_consistency(train_data_df, "Train")
    check_data_consistency(test_data_df, "Test")
    check_data_consistency(val_data_df, "Validation")

    print("Data processing completed successfully.")
    return train_data, test_data, val_data

In [9]:
train_data, test_data, val_data = generate_training_data(file_path="/content/drive/MyDrive/SJSU/DATA298/jerry_scripts.csv",
                                                     character_name="JERRY")

Loading data from file...
Filtering data for character: JERRY...
Exporting dialogues to jerry_dialogue.csv...
Data exported successfully to jerry_dialogue.csv
Processing dialogues for character: JERRY...
Compiling question-answer pairs...
Skipping non-string question at index 24156. Found type: float
Skipping non-string question at index 53142. Found type: float
Post-compilation consistency check...
Checking consistency for Q&A Pairs...
Total entries in Q&A Pairs: 14233
Q&A Pairs is complete without any missing values.
Generating training prompts...
Split data into train, test, and val sets...
Final consistency check...
Checking consistency for Train...
Total entries in Train: 11386
Train is complete without any missing values.
Checking consistency for Test...
Total entries in Test: 1423
Test is complete without any missing values.
Checking consistency for Validation...
Total entries in Validation: 1424
Validation is complete without any missing values.
Data processing completed succes

In [10]:
# Print the size of each dataset
print(f"Training data size: {len(train_data)} entries")
print(f"Test data size: {len(test_data)} entries")
print(f"Validation data size: {len(val_data)} entries")

Training data size: 11386 entries
Test data size: 1423 entries
Validation data size: 1424 entries


In [18]:
train_data[0]

{'question': 'Jerry! You missed a hell of a cockfight last night.',
 'answer': 'Then what is my check still doing up? We had a deal!',
 'training_prompt': '### Instruction: <<SYS>>\nYou are an expert at impersonating celebrities. In particular, you are\na world-leading expert at answering questions like Jerry from the hit\nsitcom Seinfield.\n<</SYS>>\n\n<<INST>>\nGenerate a response that sounds as close to what Jerry would say. I have\nincluded some relevant samples of what he actually said in the show.\n<</INST>>\n\n### Input:\nJerry! You missed a hell of a cockfight last night.\n\n### Context:\n "Jerry"!\n\n "Jerry"!\n\n Jerry.\n\n Jerry.\n\n Jerry.\n\n (to himself) Oh God! (in the microphone) huh... Hey! How you folks doing tonight? (everybody in the crowd is talking over Jerry)\n\n Absolutely. But I feel I must inform you that what happened last night was more than a simple bet.\n\n (Jerry gets up and walk to these persons) Uh... Excuse me. I couldn\'t help but notice you offered h

In [19]:
test_data[0]

{'question': "I didn't have make-up sex.",
 'answer': "In your situation the only sex you're going to have better than make-up sex is if you're dent to prison and you have a conjugal visit.",
 'training_prompt': "### Instruction: <<SYS>>\nYou are an expert at impersonating celebrities. In particular, you are\na world-leading expert at answering questions like Jerry from the hit\nsitcom Seinfield.\n<</SYS>>\n\n<<INST>>\nGenerate a response that sounds as close to what Jerry would say. I have\nincluded some relevant samples of what he actually said in the show.\n<</INST>>\n\n### Input:\nI didn't have make-up sex.\n\n### Context:\n You didn't have make-up sex? How could you not have make-up sex? I mean that's the best feature of the heavy relationship.\n\n Well, at least you probably had some, uh, pretty good make-up sex after.\n\n In your situation the only sex you're going to have better than make-up sex is if you're dent to prison and you have a conjugal visit.\n\n (walking away) Well,

In [20]:
val_data[0]

{'question': "Well, you're going over there.",
 'answer': "I don't wanna mix in everything! My guys don't know your guys. You can't just lock 'em all in the same machine together. They'll start a riot.",
 'training_prompt': "### Instruction: <<SYS>>\nYou are an expert at impersonating celebrities. In particular, you are\na world-leading expert at answering questions like Jerry from the hit\nsitcom Seinfield.\n<</SYS>>\n\n<<INST>>\nGenerate a response that sounds as close to what Jerry would say. I have\nincluded some relevant samples of what he actually said in the show.\n<</INST>>\n\n### Input:\nWell, you're going over there.\n\n### Context:\n But you're going over there for dinner tonight, right?\n\n So how come you're not going over there for dinner?\n\n Well, I'm not going over there. That's where the Lopper is.\n\n Yeah, I did. In fact I went over there.\n\n Yeah, over here.\n\n Well, where are you going?\n\n Put it over there.\n\n No the stools go over there.\n\n But that's a one

In [29]:
import pickle

def save_data_with_pickle(data, train_data, test_data, filename_prefix):
    with open(f'{filename_prefix}_train_data.pkl', 'wb') as file:
        pickle.dump(data, file)
    with open(f'{filename_prefix}_test_data.pkl', 'wb') as file:
        pickle.dump(train_data, file)
    with open(f'{filename_prefix}_val_data.pkl', 'wb') as file:
        pickle.dump(test_data, file)

save_data_with_pickle(train_data, test_data, val_data, '/content/drive/MyDrive/SJSU/DATA298/jerry/jerry')

In [30]:
def load_data_with_pickle(filename_prefix):
    with open(f'{filename_prefix}_train_data.pkl', 'rb') as file:
        train_data = pickle.load(file)
    with open(f'{filename_prefix}_test_data.pkl', 'rb') as file:
        test_data = pickle.load(file)
    with open(f'{filename_prefix}_val_data.pkl', 'rb') as file:
        val_data = pickle.load(file)
    return train_data, test_data, val_data

train_data, test_data, val_data = load_data_with_pickle('/content/drive/MyDrive/SJSU/DATA298/jerry/jerry')